In [1]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from astropy.io import fits
from astropy.time import Time
def utc2met(myUTCstring):
    UTC0=Time('2019-01-01',format='iso',scale='utc')
    datastart='2020-12-07 03:35:44.0' # valid data since when the mission starts
    datastartUTC=Time(datastart,format='iso',scale='utc')
    if isinstance(myUTCstring,(list,tuple,np.ndarray)):
        myUTC=Time(np.array(myUTCstring),format='iso',scale='utc')
        for anymyUTC in myUTC:
            assert anymyUTC>=datastartUTC,'**** ERROR: One of the UTC TIME is not valid!!! ****: '+str(anymyUTC)
        return myUTC.gps-UTC0.gps
    elif isinstance(myUTCstring,str):
        myUTC=Time(myUTCstring,format='iso',scale='utc')
        assert myUTC>=datastartUTC,'**** ERROR: One of the UTC TIME is not valid!!! ****: '+myUTCstring
        return myUTC.gps-UTC0.gps
    else:
        print('Check your input format!')
        return None        

def met2utc(myMET):
    UTC0=Time('2019-01-01',format='iso',scale='utc')
    if isinstance(myMET,(list,tuple,np.ndarray)):
        myTimeGPS=Time(np.array(myMET)+UTC0.gps,format='gps',scale='utc')
        return myTimeGPS.iso
    elif np.isscalar(myMET):
        myTimeGPS=Time(myMET+UTC0.gps,format='gps',scale='utc')
        return myTimeGPS.iso
    else:
        print('Check your input format!')
        return None

'''
def met2utc_old(myMET):
    UTC0=Time('2019-01-01',format='iso',scale='utc')
    if isinstance(myMET,(list,tuple,np.ndarray)):
        myMETsize=len(myMET)
        utc_tt_diff=np.zeros(myMETsize)+37.0
        myTimeGPS=Time(np.array(myMET)+UTC0.gps-utc_tt_diff,format='gps')
        return myTimeGPS.iso
    elif np.isscalar(myMET):
        utc_tt_diff=37.0
        myTimeGPS=Time(myMET+UTC0.gps-utc_tt_diff,format='gps')
        return myTimeGPS.iso
    else:
        print('Check your input format!')
        return None
'''

seqs = [f'{i:02}' for i in np.arange(1,26)]
CH1 = 0 # good channel
CH2 = 447

    
test_data_dir = "/ws/FormalWork/GECAM_test/data/"
daily_data_A_dir = test_data_dir+"/GECAM_A/"
daily_data_B_dir = test_data_dir+"/GECAM_B/"
trigger_data_dir = test_data_dir+"/triggers/"
burst_data_dir = test_data_dir+"/bursts/"

In [2]:
# burst data
viewt1 = -50
viewt2 = 100
binwidth = 0.064

tbins = np.arange(viewt1,viewt2+binwidth,binwidth)
bns = ['bn201209_213202','bn201209_221321','bn201209_221742']
for bn in bns:
    burst_A_data_dir = burst_data_dir + '/' + bn + '/GECAM_A/'
    burst_B_data_dir = burst_data_dir + '/' + bn + '/GECAM_B/'
    gag_evt_file = glob(burst_A_data_dir+'/gag_evt_'+bn+'_*.fits')
    gac_evt_file = glob(burst_A_data_dir+'/gac_evt_'+bn+'_*.fits')
    gbg_evt_file = glob(burst_B_data_dir+'/gbg_evt_'+bn+'_*.fits')
    gbc_evt_file = glob(burst_B_data_dir+'/gbc_evt_'+bn+'_*.fits')
    assert len(gag_evt_file) == 1
    assert len(gac_evt_file) == 1
    assert len(gbg_evt_file) == 1
    assert len(gbc_evt_file) == 1
    for j in range(2):
        grdfile = [gag_evt_file[0],gbg_evt_file[0]][j]
        fig, axes = plt.subplots(5,5,figsize=(50, 30),
                            sharex=True,sharey=False)
        hdu = fits.open(grdfile)
        trigtime = hdu['Primary'].header['BST_TIME']
        ebound = hdu['EBOUNDS'].data
        emin = ebound.field(1)
        emin = emin[CH1:CH2+1]
        emax = ebound.field(2)
        emax = emax[CH1:CH2+1]
        for i in range(25): 
            data = hdu['EVENTS'+seqs[i]].data
            t = data.field(0) -trigtime
            ch = data.field(1)
            goodindex = (ch >= CH1) & (ch <= CH2)
            t = t[goodindex]
            histvalue, histbin = np.histogram(t,bins=tbins)
            plotrate = histvalue/binwidth
            plotrate = np.concatenate(([plotrate[0]],plotrate))
            axes[i//5,i%5].plot(histbin,plotrate,drawstyle='steps')
            axes[i//5,i%5].set_xlim([viewt1,viewt2])
            axes[i//5,i%5].tick_params(labelsize=25)
            axes[i//5,i%5].text(0.05,0.85,seqs[i],fontsize=25,
                                transform=axes[i//5,i%5].transAxes)
            axes[i//5,i%5].text(0.3,0.80,(str(round(emin[0],1))
                        +'-'+str(round(emax[-1],1))+' keV'),
                        transform=axes[i//5,i%5].transAxes,fontsize=25)  
        plt.savefig('./'+bn+str(j)+'lc.png')
        plt.close()

In [3]:
utc2met('2020-12-09 21:32:02')

61248722.0

In [4]:
utc2met(1)

Check your input format!


In [5]:
np.array(['2020-11-09 21:32:02','2020-12-09 21:30:02'])

array(['2020-11-09 21:32:02', '2020-12-09 21:30:02'], dtype='<U19')

In [6]:
met2utc(61248722.0)

'2020-12-09 21:32:02.000'

In [7]:
met2utc([61248722.0,61248720.0])

array(['2020-12-09 21:32:02.000', '2020-12-09 21:32:00.000'], dtype='<U23')

In [8]:
met2utc(10000.0)

'2019-01-01 02:46:40.000'